<a href="https://colab.research.google.com/github/stevenbowler/LamdaLabsTest/blob/master/colabs/TestMedChatBotAdapted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test the MedChatBotAdapted

model located in  huggingface/stevenbowler/MedChatBotAdapted, trained with auto-train on LambdaLabs GPU
Uses bigbio/med_qa dataset for fine-tuning
See other colabs in this repo: AutoTrain_LLM to create the model

In [ ]:
!pip install -qU \
  transformers \
  sentence-transformers \
  pinecone-client \
  datasets \
  accelerate \
  einops \
  langchain \
  xformers \
  bitsandbytes \
  pypdf

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
# connect to huggingface
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import os
import getpass

hf_token = getpass.getpass('huggingface token: ')
os.environ['HF_TOKEN'] = hf_token

In [ ]:
model_name = "stevenbowler/MedChatBotAdapted"

In [ ]:
from transformers import AutoModelForCausalLM
from torch import cuda, bfloat16
import transformers

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_quant_type='nf4',
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    gradient_checkpointing=True,
    use_auth_token=hf_token
)

model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      config=model_config,
      quantization_config=bnb_config,
      device_map='auto',
      use_auth_token=hf_token
      )